In [1]:
import os
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb

import pickle

In [2]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [3]:
# df_mulliken_charges = pd.read_csv(FOLDER + 'mulliken_charges.csv')
# df_sample =  pd.read_csv(FOLDER + 'sample_submission.csv')
# df_magnetic_shielding_tensors = pd.read_csv(FOLDER + 'magnetic_shielding_tensors.csv')
# df_train = pd.read_csv(FOLDER + 'train.csv')
# df_test = pd.read_csv(FOLDER + 'test.csv')
# df_dipole_moments = pd.read_csv(FOLDER + 'dipole_moments.csv')
# df_potential_energy = pd.read_csv(FOLDER + 'potential_energy.csv')
# df_structures = pd.read_csv(FOLDER + 'structures.csv')
# df_scalar_coupling_contributions = pd.read_csv(FOLDER + 'scalar_coupling_contributions.csv')

In [ ]:
def type_score(y_val, y_pred):
    return np.log(sum(np.abs(y_val- y_pred)) / len(y_val))

In [ ]:
try_test = '20190701_dist_bond_dir_cos_train_{}.csv'
trained_model = '20190701_champs_models_lgb_{}.pickle'
submission_name = '20190702submission.csv'

In [ ]:
bond_types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHH', '3JHC', '3JHN']

In [ ]:
df_submit = pd.DataFrame()
for i, b in enumerate(bond_types):
    print('-'*10, b, '-'*10)
    df_bond = pd.read_csv(OUTPUT +try_test.format(b))
    y_pred_b =[]
    
    with open(OUTPUT + trained_model.format(b), 'rb') as f:
        models_b = pickle.load(f)

    X_test = df_bond[df_bond.columns[5:]].values  
    num_seed = len(models_b)
    for s in range(num_seed):
        gbm = models_b[s]
        y_pred_single = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        y_pred_b.append(y_pred_single)
        
    y_pred = np.array(y_pred_b).mean(axis=0)
    
    df_submit_b = pd.DataFrame([df_bond['id'], y_pred], index=['id', 'scalar_coupling_constant']).T
    
    df_submit = pd.concat([df_submit, df_submit_b], axis=0)

---------- 1JHC ----------
seed No.0, bond type 1JHC


In [ ]:
df_submit.head()

In [ ]:
df_submit.to_csv(OUTPUT + submission_name, index=False)